# Min Norm Solution for Bleed Removal

### Toy Example

In [2]:
import numpy as np
import numpy.linalg as linalg

#### Inteference Matrix (lambda)

In [88]:
lmbda = np.array([[1, 0.2, 0.1], [0.2, 1, 0.1], [0.2, 0.1, 1]])
lmbda

array([[1. , 0.2, 0.1],
       [0.2, 1. , 0.1],
       [0.2, 0.1, 1. ]])

#### Source R = lambda.S

In [101]:
R = np.random.rand(3,1000)
R

array([[0.30623236, 0.29007558, 0.32656067, ..., 0.26782798, 0.73473915,
        0.50643876],
       [0.48718847, 0.32256357, 0.95434424, ..., 0.77160613, 0.16073067,
        0.12513822],
       [0.33245031, 0.65055748, 0.49165979, ..., 0.97073451, 0.64070822,
        0.50104996]])

In [103]:
S = np.dot(lmbda, R)
S

array([[0.43691508, 0.41964405, 0.5665955 , ..., 0.51922266, 0.83095611,
        0.5815714 ],
       [0.58167997, 0.44563444, 1.06882235, ..., 0.92224518, 0.37174932,
        0.27653097],
       [0.44241562, 0.74082895, 0.65240635, ..., 1.10146072, 0.80372912,
        0.61485154]])

#### Norm Solution

In [104]:
def leastnormsoln(r, s):
    b = np.array([r-s]).T
    A = np.array([[s[1], s[2], 0, 0 ,0, 0], [0, 0, s[0], s[2], 0, 0], [0, 0, 0, 0, s[0], s[1]]])
    soln, residual, rank, singular = linalg.lstsq(A, b, rcond=-1)
    return np.array([[1, soln[0][0], soln[1][0]], [soln[2][0], 1, soln[3][0]], [soln[4][0], soln[5][0], 1]])

In [105]:
lmda = []
for i in range(R.shape[1]):
    r = R[:, i]
    s = S[:, i]
    lmda.append(leastnormsoln(r, s))

#### Different Solution for each column

In [107]:
lmda[0]

array([[ 1.        , -0.142329  , -0.10825295],
       [-0.10678206,  1.        , -0.1081264 ],
       [-0.09078098, -0.12085982,  1.        ]])

### Actual Audio (Sample from MUSDBHQ Bleeded)

In [165]:
import os
import librosa as lb
import soundfile as sf

#### Loading Bleed Sources

In [130]:
bleed_path = '/home/rajesh/Desktop/Datasets/musdb18hq_bleeded/train/A Classic Education - NightOwl/'
bvocals, fs = lb.load(bleed_path+'vocals.wav')
bbass, fs = lb.load(bleed_path+'bass.wav')
bdrums, fs = lb.load(bleed_path+'drums.wav')
bother, fs = lb.load(bleed_path+'other.wav')

In [124]:
R = np.array([bvocals, bbass, bdrums, bother])
R.shape

(4, 3780252)

#### Loading True Sources

In [126]:
clean_path = '/home/rajesh/Desktop/Datasets/musdb18hq/train/A Classic Education - NightOwl/'
vocals, fs = lb.load(clean_path+'vocals.wav')
bass, fs = lb.load(clean_path+'bass.wav')
drums, fs = lb.load(clean_path+'drums.wav')
other, fs = lb.load(clean_path+'other.wav')

In [129]:
S = np.array([vocals[:3780252], bass[:3780252], drums[:3780252], other[:3780252]])
S.shape

(4, 3780252)

#### Min Norm Solution

In [133]:
def leastnormsoln(r, s):
    b = np.array([r-s]).T
    A = np.array([[s[1], s[2], s[3], 0, 0 ,0, 0, 0, 0, 0, 0, 0], 
                  [0, 0, 0, s[0], s[2], s[3], 0, 0, 0, 0, 0, 0], 
                  [0, 0, 0, 0, 0, 0, s[0], s[1], s[3], 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 0, s[0], s[1], s[2]]])
    soln, residual, rank, singular = linalg.lstsq(A, b, rcond=-1)
    return np.array([[1, soln[0][0], soln[1][0], soln[2][0]],
                     [soln[3][0], 1, soln[4][0], soln[5][0]],
                     [soln[6][0], soln[7][0], 1, soln[8][0]],
                     [soln[9][0], soln[10][0], soln[11][0], 1]])

In [134]:
lmda = []
for i in range(R.shape[1]):
    r = R[:, i]
    s = S[:, i]
    lmda.append(leastnormsoln(r, s))

In [136]:
lmda[0]

array([[ 1.        ,  0.15573787,  0.02468029,  0.02048064],
       [-0.10313342,  1.        ,  0.07998507,  0.06637464],
       [-0.02391373,  0.11703096,  1.        ,  0.01539041],
       [-0.02336321,  0.11433679,  0.01811933,  1.        ]])

#### Getting Back Sources (Lambda Inverse)

In [139]:
def predict(r, l):
    lambda_inv = linalg.inv(l)
    soln = np.dot(lambda_inv,r)
    return soln

In [160]:
coln_soln = []
for i in range(R.shape[1]):
    r = R[:, i]
    l = lmda[i]
    coln_soln.append(predict(r, l))

In [161]:
unbleeded = np.array(coln_soln).T

In [163]:
unbleed_vocal = unbleeded[0]
unbleed_bass = unbleeded[1]
unbleed_drum = unbleeded[2]
unbleed_other = unbleeded[3]

In [167]:
sf.write('/home/rajesh/Desktop/vocal.wav', unbleed_vocal, fs)
sf.write('/home/rajesh/Desktop/bass.wav', unbleed_bass, fs)
sf.write('/home/rajesh/Desktop/drum.wav', unbleed_drum, fs)
sf.write('/home/rajesh/Desktop/other.wav', unbleed_other, fs)

In [168]:
sf.write('/home/rajesh/Desktop/bleed_vocal.wav', bvocals, fs)
sf.write('/home/rajesh/Desktop/bleed_bass.wav', bbass, fs)
sf.write('/home/rajesh/Desktop/bleed_drum.wav', bdrums, fs)
sf.write('/home/rajesh/Desktop/bleed_other.wav', bother, fs)

In [169]:
sf.write('/home/rajesh/Desktop/true_vocal.wav', vocals, fs)
sf.write('/home/rajesh/Desktop/true_bass.wav', bass, fs)
sf.write('/home/rajesh/Desktop/true_drum.wav', drums, fs)
sf.write('/home/rajesh/Desktop/true_other.wav', other, fs)

#### Average Lambda Soln

In [180]:
avg_lmda = np.average(lmda, axis=0)

In [181]:
coln_soln = []
for i in range(R.shape[1]):
    r = R[:, i]
    l = avg_lmda
    coln_soln.append(predict(r, l))

In [182]:
unbleeded = np.array(coln_soln).T

In [183]:
unbleed_vocal = unbleeded[0]
unbleed_bass = unbleeded[1]
unbleed_drum = unbleeded[2]
unbleed_other = unbleeded[3]

In [184]:
sf.write('/home/rajesh/Desktop/avg_vocal.wav', unbleed_vocal, fs)
sf.write('/home/rajesh/Desktop/avg_bass.wav', unbleed_bass, fs)
sf.write('/home/rajesh/Desktop/avg_drum.wav', unbleed_drum, fs)
sf.write('/home/rajesh/Desktop/avg_other.wav', unbleed_other, fs)

#### Max Val Lambda Soln

In [185]:
max_lmda = np.amax(lmda, axis=0)
max_lmda

array([[1.        , 0.55743289, 0.52545245, 0.5616802 ],
       [0.5277841 , 1.        , 0.66251278, 1.01806751],
       [0.63701754, 1.10782773, 1.        , 1.63168299],
       [0.64120316, 1.14406766, 0.66863615, 1.        ]])

In [186]:
coln_soln = []
for i in range(R.shape[1]):
    r = R[:, i]
    l = max_lmda
    coln_soln.append(predict(r, l))

In [187]:
unbleeded = np.array(coln_soln).T

In [188]:
unbleed_vocal = unbleeded[0]
unbleed_bass = unbleeded[1]
unbleed_drum = unbleeded[2]
unbleed_other = unbleeded[3]

In [189]:
sf.write('/home/rajesh/Desktop/max_vocal.wav', unbleed_vocal, fs)
sf.write('/home/rajesh/Desktop/max_bass.wav', unbleed_bass, fs)
sf.write('/home/rajesh/Desktop/max_drum.wav', unbleed_drum, fs)
sf.write('/home/rajesh/Desktop/max_other.wav', unbleed_other, fs)

#### Min Val Lambda Soln

In [191]:
min_lmda = np.amin(lmda, axis=0)
min_lmda

array([[ 1.        , -1.12310153, -0.39259231, -1.32638088],
       [-2.62817349,  1.        , -0.38083775, -0.86411535],
       [-0.83620876, -0.57315082,  1.        , -0.7267434 ],
       [-1.62901102, -0.55622151, -1.42703541,  1.        ]])

In [192]:
coln_soln = []
for i in range(R.shape[1]):
    r = R[:, i]
    l = min_lmda
    coln_soln.append(predict(r, l))

In [193]:
unbleeded = np.array(coln_soln).T

In [194]:
unbleed_vocal = unbleeded[0]
unbleed_bass = unbleeded[1]
unbleed_drum = unbleeded[2]
unbleed_other = unbleeded[3]

In [195]:
sf.write('/home/rajesh/Desktop/min_vocal.wav', unbleed_vocal, fs)
sf.write('/home/rajesh/Desktop/min_bass.wav', unbleed_bass, fs)
sf.write('/home/rajesh/Desktop/min_drum.wav', unbleed_drum, fs)
sf.write('/home/rajesh/Desktop/min_other.wav', unbleed_other, fs)